In [1]:
try:    importlib.reload(Jupytils)
except: import Jupytils
import itertools as it
import cvxopt
from cvxopt import matrix, solvers

showTopbar("Markov Chains", menu=[("Help", "#"), ("Home", "#")])
np.set_printoptions(precision=4, suppress=True)

<IPython.core.display.Javascript object>

In [2]:
import random, shlex


class NaiveMarkov:
    """An nth-Order Markov Chain class with some lexical processing elements."""
    def __init__(self, delim, order, n=10000):
        """Initialized with a delimiting character (usually a space) and the order of the Markov chain."""
        self.states = {}
        self.delim = delim
        self.max = n
        if order > 0:
            self.order = order
        else:
            raise Exception('Markov Chain order cannot be negative or zero.')

    def init_chain(self):
        """Helper function to generate the correct initial chain value."""
        init = []
        for i in range(self.order):
            init.append('');
        return tuple(init)

    def step(self, a, e):
        """Helper function that pops the end of tuple 'a' and tags on str 'e'."""
        d = a[1:] + (e,)
        return d

    def GetTokens(self, sample):
        if (type(sample) == str ):
            tokens = sample.split(self.delim)
        else:
            tokens = sample;
            
        return tokens;

    def learn(self, sample):
        """Adds states to the dictionary; works best with sentences."""
        prev = self.init_chain()
        tokens = self.GetTokens(sample)
        self.tokens = tokens;
        
        self.max = len(tokens)
        for t in tokens:
            if not prev in self.states:
                self.states[prev] = []
            curr = self.step(prev, t)
            self.states[prev].append(curr)
            prev = curr

    def learn_file(self, url):
        """Parses a text document sentence-wise using shlex and 'learns' each sentence."""
        f = open(url, 'r')
        s = shlex.shlex(f)
        t = s.get_token()
        sent = ''
        while t != None and t != '':
            t = t.replace('"', '').replace("'", '') \
                .replace('(','').replace(')','').replace('[','') \
                .replace(']','')
            if t.endswith('.') or t.endswith('!') or t.endswith('?'):
                sent = sent + t
                sent = sent + self.delim
                self.learn(sent)
                sent = ''
            else:
                sent = sent + t
                sent = sent + self.delim
            t = s.get_token()
        f.close()

    def query(self, n= None):
        """Queries the Markov chain with the default seed (defined by init_chain)."""
        if ( n is None): n = self.max
        tmp = self.init_chain()
        t = self.states[tmp][random.randint(0, len(self.states[tmp])-1)]
        sent = ''
        first = True
        i=0;
        while t in self.states and t != tmp and t != None:
            if not first:
                sent = sent + self.delim
            sent = sent + str(t[len(t)-1])
            t = self.states[t][random.randint(0, len(self.states[t])-1)]
            first = False
            #print(sent)
            i += 1
            if (i >= n):  
                break;
        return sent;  # + self.delim + "===" + t[len(t)-1]

    def ask(self, seed, n=None):
        """Queries the Markov chain with the seed of your choice (make sure it is an n-element list/tuple where n is the order of the chain)."""
        if ( n is None): n = self.max
        tmp = seed
        if not tmp in self.states.keys():
            tmp = self.init_chain()
        t = self.states[tmp][random.randint(0, len(self.states[tmp])-1)]
        sent = ''
        first = True
        #print(":"+ str(t), end='')
        i=0
        while t in self.states and t != tmp and t != None:
            #print('+', end='')
            if not first:
                sent = sent + self.delim
            sent = sent + str(t[len(t)-1])
            t = self.states[t][random.randint(0, len(self.states[t])-1)]
            first = False
            i += 1
            if (i > n):  
                print(len(sent), i)
                break;
                
        return sent; # + self.delim + t[len(t)-1]
    

    
class HOMVMarkov:
    """An higher order multi variate Markov Chain"""
    def __init__(self, X= None, nStates=None, order=1):
        assert order > 0, "order cannot be negative"
        self.nStates = nStates
        self.order = order
        self.X = X;

    def ComputeHOMC(self, X=None, order=1, numClasses=None):
        fS={}
        pS={}
        xHats=[]
        numClasses = numClasses if numClasses is not None else len(unique(X[0]))

        if ( X is None):
            X = self.X;
            numClasses = self.nStates
            order = self.order
        
        for x in X:
            xHats.append(self.computeXHat(x, numClasses) )

        for i in range(order):
            a1 = X[0]
            a2 = a1[i+1:]
            F=self.Freq(a1, a2, numClasses)
            fS[i] = F[0]
            pS[i] = F[1]
        
        self.fS = fS; self.pS = pS; self. xHats = xHats;
        
        return fS, pS, xHats;
    
    #---- Private stuff
    def Encode(self, y):
        l = preprocessing.LabelEncoder()
        y = l.fit_transform(y);
        return y, l.classes_

    #--- 
    # Must provide classes encoded by 0, 1, 2 etc
    def Freq(self, s1, s2 = None, numClasses=None):
        if len(s1) <= 0:
            return None;

        if (s2 is None):
            s2 = s1[1:]

        numClasses = numClasses if numClasses else max(max(s1), max(s2))+1
        F=np.zeros((numClasses, numClasses))
        for z in zip(s2,s1):
            F[(z)] += 1

        F=np.matrix(F)
        P=F.copy();
        P = P/P.sum(axis=0)
        return F, P
    

    def computeXHat(self, s, numClasses =None):

        numClasses = numClasses if numClasses else max(s)+1
        xHat=np.zeros(numClasses)
        t=pd.Series(s).value_counts() 
        for i,j in t.items():    
            xHat[i] =j
        ret = xHat/xHat.sum()

        ret = np.array([ret]).T

        return ret;

    #Display thr matrix
    def Matdisplay(*M, names=None):
        s = ""
        if (names is None):
            names = ["" for i in range(len(M)) ]
        for i, m in enumerate(M):
            s+= LA.M(m, name=names[i], call_display=False, showdim=False);
        display(Math(s))

    def Mdisplay(fS, pS, xHats):

        g = ["\hat{{F}}^{{{}}}".format(_) for _ in sorted(fS.keys())]
        v = [_[1] for _ in sorted(fS.items())]
        Matdisplay(*v, names=g)

        g = ["\hat{{P}}^{{{}}}".format(_) for _ in sorted(pS.keys())]
        v = [_[1] for _ in sorted(pS.items())]
        Matdisplay(*v, names=g)

        g = ["\hat{{x}}_{}".format(i+1) for i in range(len(xHats))]
        Matdisplay(*xHats, names=g)

    
    def PrepareMatrices(self):
        s=len(self.X)
        n= self.nStates
        ls = list(it.product(range(s), repeat=2))
        numParams = self.order

        numRows = (2*s + numParams + 2*s*n)
        # numRows= 
        # 1. 2*s for sum of lambdas equal to 1 - we need two (1) for >= 1 (2) <= 1 plus
        # 2. lambda)_jk >=0, therefore  -1 * lamda)_jk <= 0 -> 2 for each lambda
        # 3. 2 for each s * numClasses (n) , the set of   lambdas

        #-1. ---- Prepare A
        A=[]
        b=[]

        for i in range(s):
            b += [1,-1]

            c2=[1 for _ in range(numParams+1)]
            c2[-1]= 0;  

            A += c2;
            A += [_ * -1 for _ in c2];

        A = np.array(A).reshape(int(len(A)/(numParams+1)),numParams+1, order='C')

        #-2. ---- Prepare A -> lambda's must be non negative
        for i in range(numParams):
            b += [0]

            c2=[0 for _ in range(numParams+1)]
            c2[i]= -1;  
            A = vstack((A, c2))

        #-3. ---- Prepare A
        i=0

        x=xHats[i].flatten().tolist()
        x1 = [-1 * _ for _ in x]
        b += x1
        b += x

        j=np.zeros((n*(numParams+1)))
        j=j.reshape(n,numParams+1)
        j[:,-1]=1
        for k in range(numParams):        
            pd1 = pS.get(k)
            xh1 = xHats[0]
            bb1 = pd1 * xh1
            j[:,k] = bb1.flatten()

        A = vstack((A, j*-1))
        j[:,-1]=-1
        A = vstack((A, j))

        # =>> Compute C

        c= [0. for _ in range(numParams+1)]
        c[-1] = 1

        A = vstack((A, [_*-1 for _ in c]))
        b.append(0)

        self.c = c;
        self.A = A;
        self.b = b;
        return c,A,b;

    def Solve(self):
        self.c1=matrix(self.c)
        self.A1=matrix(self.A)
        self.b1=matrix(self.b)
        
        self.sol=solvers.lp(self.c1, self.A1, self.b1)
        self.p = self.sol['x']

        return self.sol;
    
    def Predict(self, Ct):
        sol = np.matrix ([0. for i in xHats[0]]).T
        if ( type(Ct[0]) == list):
            for i in range(len(Ct)):
                Ct[i] = np.matrix(Ct[i]).T
            
        for i in range(hm.order):
            Qx =  hm.pS[i] * Ct[i]
            Qx1=  Qx * hm.p[i]
            sol += Qx1
        self.sol = sol
        
        return sol;
    
def metrics(a, b, printOut=True):
    n=0;
    t=0;
    z = zip(a,b)
    correctClass=defaultdict(int)
    totalClass=defaultdict(int)
    
    for c in z:
        totalClass[c[0]] += 1;

        if(c[0] == c[1]):
            correctClass[c[0]] += 1;
            t+= 1
        n += 1

    if (printOut):
        print("{}\n{}".format(a[0:50], b[0:50]))
        print("Total %d, correct %d, acc: %3.2f"%(n,t,t/n))
        for i,c in totalClass.items():
            acc = correctClass[i]/c
            print("class: {} total: {}, correct: {}, acc: {}".format(i, c, correctClass[i], acc))

    return n, t, totalClass


In [15]:
# Paper Example
os1=[int(_) for _ in '1 1 2 2 1 3 2 1 2 3 1 2 3 1 2 3 1 2 1 2'.split()]

s1=[int(_)-1 for _ in os1 ]
numClasses = len(unique(s1))

X=[s1]

In [5]:
pefs='''1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 1 2 1 1 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 2 1 2 1 1 2 1 2 1 1 1 1 1 1 2 1 2 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 2 1 1 1 1 1 1 2 2 1 1 2 2 1 2 1 1 1 2 2 1 1 1 2 2 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 2 2 2 2 1 2 1 1 1 1 2 2 2 1 1 2 2 1 1 1 2 1 1 2 1 1 1 2 2 1 2 2 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 2 2 2 1 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 1 2 2 1 2 1 1 2 2 2 1 1 2 2 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 2 2 1 1 2 1 2 1 2 1 1 1 1 1 1 1 1 2 1 1 1 2 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 2 1 1 1'''
os1=[int(_) for _ in pefs.split()]
s1=[int(_)-1 for _ in os1 ]
numClasses = len(unique(s1))

X=[s1]

In [71]:
m=NaiveMarkov(' ',2)
m.learn(s1)
q=m.query()
#print("==>", len(pefs), len(q), q)
r = m.ask(tuple(''.split()), 900)
t = [int(_) for _ in m.GetTokens(r)]

metrics(s1, t)

1801 901
[0, 0, 1, 1, 0, 2, 1, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1]
[0, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 0, 2, 1, 0, 1, 2, 0, 1, 2, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 1, 2, 0, 1, 0, 2, 1, 0, 1, 2, 0]
Total 20, correct 7, acc: 0.35
class: 0 total: 8, correct: 4, acc: 0.5
class: 1 total: 8, correct: 2, acc: 0.25
class: 2 total: 4, correct: 1, acc: 0.25


(20, 7, defaultdict(int, {0: 8, 1: 8, 2: 4}))

In [17]:
order=2
m= len(unique(s1))
hm= HOMVMarkov(X,m, order)
fS,pS, xHats = hm.ComputeHOMC(X,order)
c,A,b = hm.PrepareMatrices()
sol = hm.Solve()

soltn = sum(np.array(c) * sol['x'])
params = np.array([_ for _ in sol['x'].T])
print(params, sum(params[0:2]), "solution=", soltn)

#print(A.shape, numRows, numParams)
print(A,b,c);

m= len(unique(s1))
ser = s1;
pdt = []

for j in range(len(ser) - order):
    XX=[0 for _ in range(m)]
    Xt=[]
    for _ in range(order):
        xx = XX.copy()
        Xt.append(xx)
        
    for i in range(order):
        Xt[i][ser[j+i]]=1
        
    #print(Xt, end='')

    r=hm.Predict(Xt)
    #print(Xt, r)
    mm=[i for i in range(m) if r[i] == max(r)]
    pdctd = mm[random.randint(0,len(mm)-1)]
    pdt.append(pdctd)

pdt.insert(0,s1[1])
pdt.insert(0,s1[0])
metrics(s1, pdt)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.1916e-18 -1.1102e-16  1e+01  2e+00  7e+00  1e+00
 1:  8.2336e-02  1.0361e-01  4e-01  2e-01  7e-01  1e-01
 2:  2.5887e-02  3.1648e-02  9e-02  7e-02  2e-01  4e-02
 3:  2.0190e-02  2.1674e-02  2e-02  2e-02  6e-02  1e-02
 4:  2.7965e-02  2.7839e-02  5e-03  4e-03  1e-02  1e-03
 5:  2.8561e-02  2.8560e-02  6e-05  5e-05  1e-04  2e-05
 6:  2.8571e-02  2.8571e-02  6e-07  5e-07  1e-06  2e-07
 7:  2.8571e-02  2.8571e-02  6e-09  5e-09  1e-08  2e-09
Optimal solution found.
[ 1.      0.      0.0286] 1.00000000214 solution= 1.02857142962
[[ 1.      1.      0.    ]
 [-1.     -1.      0.    ]
 [-1.      0.      0.    ]
 [ 0.     -1.      0.    ]
 [-0.3714 -0.3357 -1.    ]
 [-0.4071 -0.4357 -1.    ]
 [-0.2214 -0.2286 -1.    ]
 [ 0.3714  0.3357 -1.    ]
 [ 0.4071  0.4357 -1.    ]
 [ 0.2214  0.2286 -1.    ]
 [-0.     -0.     -1.    ]] [1, -1, 0, 0, -0.4, -0.4, -0.2, 0.4, 0.4, 0.2, 0] [0.0, 0.0, 1]
[0, 0, 1, 1, 0, 2, 1, 0, 1, 2, 0, 1, 2, 0, 1, 2,

(20, 11, defaultdict(int, {0: 8, 1: 8, 2: 4}))

In [10]:
np.array(c) * list(sol['x'])

array([ 0.    ,  0.    ,  0.0002])

In [ ]:
Product A: 6 6 6 6 2 6 2 6 2 2 6 2 6 6 2 6 2 4 4 4 5 6 6 1 2 2 6 6 6 2 6 2 6 6 2 6 2 2 6 2 1 2 2 6 6 6 2 1
    2 6 2 6 6 2 2 6 2 2 2 6 2 6 2 2 2 2 2 6 2 2 6 6 6 6 1 2 2 6 2 2 2 2 6 2 2 2 2 3 3 2 
    3266662626626266266223433131216166166262622266162612162622226616622622234446461661666616222666626 
    6226262226222666632262222226262226226626662223334166166161666616662122222236666626

<hr>
## Examples (Other)

In [ ]:
## Example From [1]

price='''
5 5 5 5 4 5 3 5 3 3 4 2 5 5 3 1 1 1 3 3 4 1 5 1 1 3 3 2 5 1 5 1 5 5 5 5 2 1 4 1 1 1 2 4 5 5 1 4 2 4 1 3 4 2 2 5 2 2 5 5 
2 5 4 4 4 2 2 5 2 2 5 5 5 5 3 2 2 5 4 5 2 4 5 5 4 1 1 1 2 2 3 2 4 5 5 5 2 5 2 5 5 2 4 2 5 5 2 5 5 1 2 3 4 3 3 1 3 1 4 3
5 4 5 5 4 5 5 2 5 2 5 2 2 3 5 5 3 5 2 5 4 2 1 5 2 5 2 2 2 2 5 5 4 5 5 2 2 5 2 2 2 3 4 4 4 5 4 5 1 5 5 1 3 5 5 5 1 5 2 2
2 5 5 5 5 2 4 5 2 2 5 2 5 2 2 2 4 2 2 2 4 5 5 5 3 2 2 5 2 5 4 4 4 5 3 3 5 3 1 1 4 2 2 5 5 2 5 5 5 2 5 5 3 5 5 4 1 5 5 1
5 5 1 5 1 5 5 5 5 1 5 5 5 2 1 2 5 2 5 5 2 3 5 5 5 5 5 2 5''' 

sales='''
5 1 1 1 1 2 2 3 3 3 4 3 2 2 5 1 5 5 1 2 3 3 2 3 3 2 2 1 1 5 2 3 3 3 3 2 3 1 2 1 1 5 2 2 5 5 2 3 4 3 4 2 2 1 5 5 1 5 1 5
5 1 1 5 3 3 3 3 3 3 4 3 3 5 1 5 5 1 1 5 1 5 5 1 5 5 1 5 5 1 5 1 5 2 3 3 3 3 3 3 3 5 1 5 5 1 5 1 5 5 5 5 1 5 1 1 5 3 3 3
3 3 3 5 2 2 5 1 1 1 5 1 5 1 1 1 1 1 1 1 1 1 1 1 1 2 5 3 4 4 4 4 1 3 5 5 1 5 5 1 1 5 1 1 1 1 5 1 2 1 1 2 5 2 1 1 2 3 3 3
3 4 4 3 2 2 5 1 5 1 1 1 5 2 1 1 1 1 4 4 3 3 3 3 2 5 1 5 5 1 5 1 5 1 1 2 2 3 3 4 3 3 1 1 1 2 1 1 5 1 1 1 5 1 1 1 1 1 1 1
2 3 3 1 1 4 3 1 3 2 1 1 1 1 1 5 5 1 5 1 5 1 1 1 1 1 1 1'''


p = [int(c)-1 for c in price.replace('\n', ' ').strip().split()]
s = [int(c)-1 for c in sales.replace('\n', ' ').strip().split()]

numClasses = 5
X=[p[:-6]]

fs, ps, xHats = ComputeMMC(X, numClasses = numClasses)
Mdisplay(fs, ps, xHats)

## References:

1. <a src="https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwjM66WC3JbXAhVil1QKHTj1CakQFggtMAA&url=http%3A%2F%2Fwww.ccsenet.org%2Fjournal%2Findex.php%2Fmas%2Farticle%2FviewFile%2F6040%2F4874_1_1_1&usg=AOvVaw0fZh5XmtYpF14lVZRfr6aZ">  Application of Markov Chains to Analyze and Predict the Time Series </a>

2. A multivariate Markov chain model for categorical data sequences and its applications in demand predictions
   Wai‐Ki Ching  Eric S. Fung  Michael K. Ng
    
3. Higher-order multivariate Markov chains and their applications; Wai-Ki Ching , Michael K.Ng , Eric S.Fungb
    

In [ ]:
1